<a href="https://colab.research.google.com/github/Arander/CasosDiarios/blob/main/COVID_Casos_CHILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly==4.14.3

     |████████████████████████████████| 13.2MB 296kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
import datetime
today = datetime.datetime.now()

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px


In [6]:
casos_chile = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/raw/master/output/producto13/CasosNuevosCumulativo.csv', index_col="Region")


In [7]:
#Se transpone la tabla para operar con las fechas como índice y no como columna
casos = casos_chile.T
#Al parecer las fechas no son reconocidas como Datetime, entonces definimos expresamente su formato
casos.index = pd.to_datetime(casos.index, format="%Y-%m-%d")
casos.index = [d.to_pydatetime() for d in casos.index]
#Desplegamos en pantalla para comprobar estructura
casos


Region,Arica y Parinacota,Tarapacá,Antofagasta,Atacama,Coquimbo,Valparaíso,Metropolitana,O’Higgins,Maule,Ñuble,Biobío,Araucanía,Los Ríos,Los Lagos,Aysén,Magallanes,Total
2020-03-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2020-03-05,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2020-03-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2020-03-07,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-19,90.0,149.0,197.0,95.0,246.0,598.0,2129.0,310.0,373.0,218.0,939.0,527.0,369.0,315.0,9.0,40.0,6604.0
2021-03-20,101.0,125.0,237.0,85.0,257.0,695.0,2216.0,349.0,471.0,153.0,970.0,664.0,299.0,392.0,25.0,45.0,7084.0
2021-03-21,69.0,159.0,176.0,110.0,192.0,693.0,2315.0,328.0,413.0,229.0,901.0,475.0,359.0,364.0,17.0,36.0,6836.0
2021-03-22,60.0,155.0,219.0,82.0,217.0,575.0,1903.0,283.0,498.0,118.0,856.0,540.0,303.0,303.0,16.0,27.0,6155.0


PRIMER GRAFICO CON **MATPLOTLIB** (commented to preserve)

In [8]:
x = '''plt.figure(figsize=(30,10))
dias = 7
casos['Total_Movil'] = casos['Total'].rolling(dias).mean()
casos['RM_Movil'] = casos['Metropolitana'].rolling(dias).mean()
plt.plot(casos.Total_Movil, label='TOTAL PAÍS')
plt.plot(casos.RM_Movil, label='REGIÓN METROPOLITANA')
plt.title("Casos Diarios COVID - Total Chile y Región Metropolitana (Promedio Móvil "+ str(dias) +" días)", size=20)
plt.grid(axis='y')
plt.legend(loc='lower center', shadow=True, ncol=2)
plt.show()
'''

# **USO DE PLOTLY 4.14**

GRAFICO CON PROMEDIOS MOVILES

In [9]:
locale.getlocale()

('en_US', 'UTF-8')

In [10]:
#Intentamos crear promedios móviles para todas las regiones
dias = 7 
for c in casos.columns[0:17]:
  casos[str(c)+'_Movil'] = casos[str(c)].rolling(dias).mean()
fig = px.line(casos, x=casos.index, y=casos.columns[17:34],title="Casos Diarios COVID por región (Promedio Móvil "+
              str(dias) +" días)<br>"+"Actualización: "+str(casos.index[-1].strftime('%d de %B, %Y')),
labels=dict(index="", value="Casos", variable="Región"),color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.for_each_trace(lambda t: t.update(name=t.name.replace("_Movil", "")))
#fig.update_xaxes(tickformat='%d-%B-%Y')
fig.show(locale = 'es')

GRÁFICO DE CASOS POR 100 MIL HABITANTES PARA CADA REGION

In [11]:
poblacion = {'Arica y Parinacota':226068,'Tarapacá':330558,'Antofagasta':607534,'Atacama':286168,'Coquimbo':757586,'Valparaíso':1815902,'Metropolitana':7112808,'O’Higgins':914555,
'Maule':1044950,'Ñuble':480609,'Biobío':1556805,'Araucanía':957224,'Los Ríos':384837,'Los Lagos':828708,'Aysén':103158,'Magallanes':166533,'Total':17574003}
for c in casos.columns[17:34]:
  casos[str(c)+'_x'] = casos[str(c)]/poblacion[c[:-6]]*100000
fig = px.line(casos, x=casos.index, y=casos.columns[34:51],title="Casos Diarios COVID por región (Promedio Móvil "+ str(dias) +" días) - Por 100.000 habitantes<br>"+
              "Actualización: "+str(casos.index[-1].strftime('%d/%m/%Y')),
              labels={'index':"", 'value':"Casos por 100.000 habitantes", 'variable':"Región"},
              color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.for_each_trace(lambda t: t.update(name=t.name.replace("_Movil_x", "")))
fig.show()